# PROJECT LIVE CORP

Vous êtes un jeune entrepreneur en Freelance spécialisé en machine learning. Un de vos « ami » vous a recommandé auprès d’une société nommé Live Corp. Live Corp recherche un Freelance pour lancer leur entreprise de prédiction de maladie. Cependant, cette startup commence à partir de rien, ils ont clairement besoin de vous. Ils ont tellement bien vendu le projet que les investisseurs ont donnée plusieurs milliards d’euros. Vous avez eu la CEO lors d’un entretien téléphonique :

« Voilà ce que notre équipe veut, nous souhaitons un système permettant de se connecter à internet et développer du python sur un serveur. Comme ça s’est sécurisé, on veut aussi que vous nous créer un script de veille numérique. Enfin, nous voulons une petite démonstration de prédiction du cancer du sein, nous avons besoin de mobiliser nos investisseurs pour le prochain onboarding. »

Par chance le stagiaire en communication marketing développement informatique et comunity manager vous a envoyé un mail, avec des demandes plus claires.

« Salut,
Tu as dû voir que Lucie F. n’a pas été très claire sur le sujet. J’ai besoin d’un Jupyter Hub avec PIP et sklearn (Tensorflow aussi si tu arrives à l’installer). Nous voulons plusieurs comptes pour notre prochaine arrivé de stagiaires et d’alternants. Lucie veut aussi que tu récupère les données du Flux RSS principale de l’Agence France Presse, c’est pour notre futur jeu de données, (on veut tout au format csv). De plus nous souhaitons aussi une petite classification des données en 3 sous-groupes science, économie et société. Je crois qu’elle t’a dit qu’elle voulait faire une prédiction sur les cas possible de cancer du sein. Comme nos investisseurs sont majoritairement des assurances, ils veulent savoir si on peut prédire le cancer du sein et faire en sorte de faire payer plus cher les assurés selon leur diagnostique médicale. Si tu as tout terminé avant la fin de semaine, tu peux faire une analyse de prédiction sur des gènes pour voir si les assurés sont malades.»


In [97]:
# !pip3 install feedparser
#!pip3 install spacy
!pip install spacy
!python -m spacy download fr_core_news_lg


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


2021-09-21 10:32:08.036511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-21 10:32:08.036570: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [94]:
#import
import feedparser
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import requests
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from bs4 import BeautifulSoup

In [67]:
# récupération d'un flux rss
def get_rss(rss_urls):
    columns = ['Titre','Résumé', 'Liens']
    df = pd.DataFrame(columns = columns )
    for url in rss_urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            try:
                article = pd.DataFrame([[entry.title,entry.summary,entry.link]] , columns = columns)
                df = pd.concat([df , article], ignore_index = True)
            except:
                error = url
    return df

In [74]:
#Création du dataset d'entrainement du model
urls_rss = [ 'https://www.caminteresse.fr/sciences/feed/',
             'https://www.7sur7.be/economie/rss.xml',
             'https://www.bfmtv.com/rss/societe/',
            'https://partner-feeds.20min.ch/rss/20minutes/science-et-nature',
            'https://www.7sur7.be/sciences/rss.xml',
            'https://www.courrierinternational.com/feed/category/6680/rss.xml',
            'https://www.europe1.fr/rss/faits-divers.xml',
            'https://www.europe1.fr/rss/societe.xml',
            'https://www.francetvinfo.fr/societe.rss',
            'https://www.lavenir.net/rss.aspx?foto=1&intro=1&section=info&info=12e1a2f4-7e03-4cf1-afec-016869072317',
            'https://www.lexpress.fr/rss/societe.xml',
            'https://institut-economie-circulaire.fr/feed/',
            'https://www.etudes-et-analyses.com/html/rss/eap/rss.xml',
            'https://www.20minutes.fr/feeds/rss-economie.xml',
            'https://www.abcbourse.com/rss/displaynewsrss'
            
]

dataset_train = get_rss(urls_rss)

In [ ]:
#Fonction de nettoyage
def get_cleaned_text(text_brut):
    txt = str(text_brut)
    #passe tout en minuscule
    txt = txt.lower()
    #Trouve la racine des mots
    spacy_lemmatizer = spacy.load('fr_core_news_md')
    tokens = spacy_lemmatizer(' '.join(txt))
    cleaned_txt = [token.lemma_ for token in tokens if len(token)> 3]
#     #suppression de tout ce qui n'est pas des lettres
#     txt = re.sub('[^A-Za-z]', ' ', txt)
#     #Supprime la ponctuation
# #     translator = str.maketrans('', '', string.punctuation)
# #     txt = txt.translate(translator)
#     #Supprime les espaces
#     txt = " ".join(txt.split())
    
#     #Separe les mots 
#     tokenized_txt = word_tokenize(txt)
#     #Supprime les mots vides
#     stop_words = set(stopwords.words("French"))
#     sw = ['href','https','post','caminteresse','nofollow']
#     for word in sw:
#         stop_words.add(word)
#     filtered_text = [word for word in tokenized_txt if word not in stop_words]
#     #Trouve la racine des mots
#     spacy_lemmatizer = spacy.load('fr_core_news_lg')
#     tokens = spacy_lemmatizer(' '.join(filtered_text))
#     cleaned_txt = [token.lemma_ for token in tokens if len(token)> 3]        
    
#     lemmatizer = WordNetLemmatizer()
#     lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in filtered_text]
#     cleaned_txt=[word for word in lemmas if len(word)> 3]
    cleaned_txt = ' '.join(cleaned_txt)
    return cleaned_txt

In [101]:
dataset_train['Résumé'][0]
txt_clean = get_cleaned_text(dataset_train['Résumé'][0])
txt_clean

'tout fauxquelque annes plus tard journal current literature rvl canular lech arbre exister poque public friand rcits explorateur livingston brazza marqu esprit souligne jean buard docteur science information science quest larbre anthropophage rsquo arbre anthropophage appeared first intresse curiosit continu'

In [102]:
i = 0
list_txts = []
for i in  range(len(dataset_train['Résumé'])):
    titre_cleaned = get_cleaned_text(dataset_train['Titre'][i])
    resume_cleaned = get_cleaned_text(dataset_train['Résumé'][i])
    txt_cleaned = titre_cleaned+' '+resume_cleaned
    list_txts.append(txt_cleaned)
df_train = pd.DataFrame(list_txts)
df_train

,0
0,arbre anthropophage tout fauxquelque annes plu...
1,flash curiosit fourmi plus rapide monde dcouve...
2,voir vraiment filer avant mourir dcouvrez pass...
3,grand myst matir noir salon table canap quelqu...
4,quoi constitu empreindre digital matire mlange...
5,pouvoir utiliser smartphone temps orage princi...
6,aveugle voir noir contrairement pouvoir penser...
7,bact rie devoir apparaître quelque milliard an...
8,flash curiosit fourmi plus rapide monde dcouve...
9,tourisme spatial quand plaisir quelque polluer...


In [103]:
#Affichage
# df_train
#enregistrement du dataset
df_train.to_csv('train_dataset.csv')

RSS AFP :https://www.afp.com/fr/actus/afp_actualite/792,31,9,7,33/feed
        https://www.afp.com/fr/actus/afp_communique/all/feed
            https://feed.ausha.co/blLD6S04rz64
                https://rss-bridge.lamop.fr/?action=display&bridge=Twitter&context=By+username&u=afpfr&format=Mrss
                
 economie : https://www.7sur7.be/economie/rss.xml
 
 sciences : https://www.caminteresse.fr/sciences/feed/
 
 société : https://www.bfmtv.com/rss/societe/

In [59]:
#Création du dataset de l'AFP
afp_dataset = get_rss(['https://www.afp.com/fr/actus/afp_actualite/792,31,9,7,33/feed'])
afp_dataset
#enregistrement du dataset
# afp_dataset.to_csv('afp_dataset.csv')

,Titre,Résumé,Liens
0,Les 10 vidéos les plus populaires sur les chai...,,https://www.afp.com/fr/node/3752875
1,"Le 3 octobre, « Les années argentiques » : 1èr...",<p>L'Agence France-Presse met&nbsp;aux enchère...,https://www.afp.com/fr/node/3752836
2,Le photographe AFP Josh Edelson finaliste des ...,<p>Sa couverture impressionnante des “mégafeux...,https://www.afp.com/fr/node/3752871
3,L’AFP est sélectionnée dans 2 catégories aux A...,<p>Elue “Agence de presse de l’année” en&nbsp;...,https://www.afp.com/fr/node/3752866
4,Les 10 vidéos les plus populaires sur les chai...,,https://www.afp.com/fr/node/3752860
5,Les 10 vidéos les plus populaires sur les chai...,,https://www.afp.com/fr/node/3752844
6,Les 10 vidéos les plus populaires sur les chai...,<p> </p>,https://www.afp.com/fr/node/3752838
7,Les 10 vidéos les plus populaires sur les chai...,,https://www.afp.com/fr/node/3752827
8,Les 10 vidéos les plus populaires sur les chaî...,,https://www.afp.com/fr/node/3752821
9,Les 10 vidéos les plus populaires sur les chaî...,<p> </p>,https://www.afp.com/fr/node/3752818


In [60]:
# # Récupération du deernier feed (dernier bulletin CERT-FR)
# for i in range(0, len(news_feed.entries)):
#     if i == (len(news_feed.entries)-1):
#         print("Alert: {} \nLink: {}".format(news_feed.entries[0]['title'], news_feed.entries[0]['id']))


In [104]:
def get_article():
    URL = "https://www.afp.com/fr/node/3752836"
    userAgent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.74 Safari/537.36 Edg/79.0.309.43"
    headers = {"User-Agent": userAgent }
    page_scrapped = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page_scrapped.content, 'html.parser')
    print (soup)
#     annonces=soup.find_all("div", class_="block-result-search-bottom")
#     animalName=[]
#     refugeName=[]
#     ind=[]
#     cpt=1
#     for annonce in annonces:
#         animalName.append(annonce.h3.a.string)
#         refugeName.append(annonce.find("span" , class_="refuge-name").a.contents[0].string)
#         ind.append(cpt)
#         cpt+=1
#     data = {
#         "Animal Name": pd.Series(animalName, index=ind) ,
#         "Refuge Name" : pd.Series(refugeName, index=ind)
#     }
#     df = pd.DataFrame(data)
#     return df
#tests
#get_article()
# article = get_article()
# article


In [ ]:
# Création d'une instance
news_feed = feedparser.parse('https://www.cert.ssi.gouv.fr/alerte/feed/')

# Propriétés du flux
print(news_feed.feed.keys())

# Titre du flux
print("Feed Title:", news_feed.feed.title) 

# Sous-titre du flux
print("Feed Subtitle:", news_feed.feed.subtitle)

# Lien du flux
print("Feed Link:", news_feed.feed.link, "\n")

# Propriétés de chaque item du flux
print(news_feed.entries[0].keys())

for entry in news_feed.entries:
    print(f"{entry.title} --> {entry.link}")
    
# Récupération du deernier feed (dernier bulletin CERT-FR)
for i in range(0, len(news_feed.entries)):
    if i == (len(news_feed.entries)-1):
        print("Alert: {} \nLink: {}".format(news_feed.entries[0]['title'], news_feed.entries[0]['id']))